In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
from Onsager_calc_db import *
import onsager.crystal as crystal
from states import *
from stars import *
from vector_stars import *
from representations import *
import time
import pickle
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt
import scipy.stats as spstats

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3-py3.6.egg/onsager/cluster.py:835: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
# make a BCC lattice
# We'll modify the jumpnetwork to keep only the 60 degree reorientational jumps.
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = crystal.Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110]
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = dbStates(Fe, 0, family)
mdbcontainer_fe = mStates(Fe, 0, family)
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01)

In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to closest distance

def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
#     if np.allclose(jnet0_indexed[jt][0][1], z):
#         continue
    indices.append(jt)

ind_sort = sorted(indices, key=sortkey)[:3]
len(ind_sort)

3

In [5]:
for ind in ind_sort:
    jmp = jnet0[ind][0]
    print(jmp)
    print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	dumbbell : (i, or) index = 3, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 2, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = 1

[-0.14155  0.14155  0.14155]

Jump object:
Initial state:
	dumbbell : (i, or) index = 4, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 1, lattice vector = [0 0 0]
Jumping from c1 = -1 to c2 = 1

[0. 0. 0.]

Jump object:
Initial state:
	dumbbell : (i, or) index = 3, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 3, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = -1

[-0.14155  0.14155  0.14155]



In [6]:
pdbcontainer_fe.iorlist

[(0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([-0.17819091,  0.        , -0.17819091])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([-0.17819091,  0.17819091,  0.        ])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01]))]

In [7]:
# take the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the paper
# They are: First and second lowest transition jumps, 60 deg on-site rotation, 90-deg onsite rotation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want only the rotational jumps as listed.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
indices_rot = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        jmp = jlist[0]

        or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
        or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
        
        if np.allclose(np.dot(or1,or2)/(np.linalg.norm(or1)*np.linalg.norm(or2)), np.cos(np.pi/3.)):
#             print("got 60 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        elif np.allclose(np.dot(or1,or2), 0.):
#             print("got 90 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        else:
            continue
    indices2.append(jt)
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:2]
indices2all.append(indices_rot[0]) # we'll ignore the 180 degree rotation
print(indices2all, indices_rot)

[12, 14, 18] [18, 19]


In [9]:
for index in indices_rot:
    jmp = jnet2[index][0]
    print(jmp)
    # get the lattice vector
    print(np.dot(mdbcontainer_fe.crys.lattice, jmp.state1.db.R), mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1])
    print(np.dot(mdbcontainer_fe.crys.lattice, jmp.state2.db.R), mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1])
    print()

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 6, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = 1
[0. 0. 0.] [ 1.78190909e-01  1.78190909e-01 -1.01947883e-17]
[0. 0. 0.] [ 0.17819091 -0.17819091  0.        ]

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 9, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = 1
[0. 0. 0.] [ 1.78190909e-01  1.78190909e-01 -1.01947883e-17]
[0. 0. 0.] [0.17819091 0.         0.17819091]



In [10]:
# check if we have the correct type of jumps
for ind in indices2all:
    jmp = jnet2[ind][0]
    print(jmp)
#     print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 6, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [ 0 -1  0]
	dumbbell : (i, or) index = 2, lattice vector = [ 0 -1  0]
Jumping from c1 = 1 to c2 = 1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 6, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [ 0 -1  0]
	dumbbell : (i, or) index = 4, lattice vector = [ 0 -1  0]
Jumping from c1 = 1 to c2 = 1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 6, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = 1



In [11]:
for tup in mdbcontainer_fe.iorlist:
    print(tup)

(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-0.17819091,  0.        , -0.17819091]))
(0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01]))
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([-1.78190909e-01, -1.78190909e-01,  1.01947883e-17]))
(0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01]))
(0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01]))


In [12]:
# take only the lowest displacement jump
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])
len(jset0new[0]), len(jset2new[0])

(3, 3)

In [13]:
# re-initialize the calculator.
# Now, we construct the Onsager calculator with these non-local jump sets.
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.26,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
initializing NN
built shell 1: time - 0.07075095176696777
built shell 2: time - 1.5902142524719238
grouped states by symmetry: 1.0742676258087158
built mixed dumbbell stars: 0.0005297660827636719
built jtags2: 0.0006039142608642578
built mixed indexed star: 0.007817745208740234
building star2symlist : 9.250640869140625e-05
building bare, mixed index dicts : 0.00018858909606933594
2NN Shell initialization time: 3.7397167682647705

generating thermodynamic shell
built shell 1: time - 0.03887534141540527
built shell 2: time - 1.6560111045837402
built shell 3: time - 6.856255531311035
grouped states by symmetry: 5.654573917388916
built mixed dumbbell stars: 0.0004687309265136719
built jtags2: 0.0006670951843261719
built mixed indexed star: 0.005646705627441406
building star2symlist : 0.00011610984802246094
building bare, mixed index dicts : 0.00018715858459472656
thermodynamic shell generated: 19.74522352218628
Total number of states in Thermodynamic Sh

In [14]:
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# We assume that association and dissociation occurs via Johnson mechanism.

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[20]


In [15]:
len(onsagercalculator.jnet1)

163

In [16]:
# check if we have the correct jump
print(jnet43[ind_sort43[0]][0])
print(jnet43_indexed[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 4, lattice vector = [ 0  0 -1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Jumping from c1 = -1 to c2 = -1
((13, 0), array([-0.14155, -0.14155,  0.14155]))


In [17]:
print(onsagercalculator.kinetic.complexStates[jnet43_indexed[ind_sort43[0]][0][0][0]])
print(onsagercalculator.kinetic.mixedstates[jnet43_indexed[ind_sort43[0]][0][0][1]])

SdPair(i_s=0, R_s=array([0, 0, 0]), db=dumbbell(iorind=4, R=array([ 0,  0, -1])))
SdPair(i_s=0, R_s=array([0, 0, 0]), db=dumbbell(iorind=0, R=array([0, 0, 0])))


In [18]:
print(onsagercalculator.pdbcontainer.iorlist[jnet43[ind_sort43[0]][0].state1.db.iorind])
print(onsagercalculator.mdbcontainer.iorlist[jnet43[ind_sort43[0]][0].state2.db.iorind])

(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17]))
(0, array([ 0.        , -0.17819091,  0.17819091]))


In [19]:
# extract the network
onsagercalculator.regenerate43(ind_sort43)

In [20]:
with open('FeMn_Onsg.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)